In [44]:
import requests
import json
import pandas as pd
import numpy as np
from functools import reduce
from datetime import datetime
import sys
sys.path.append('/Users/MikeTodisco/opt/anaconda3/lib/python3.7/site-packages')

from draft_kings import Sport, Client
from tqdm.notebook import tqdm


In [2]:
contests = Client().contests(sport=Sport.GOLF)

def get_draft_group_id(contest_obj, contest_type_id=84):
    draft_groups = contest_obj.draft_groups
    draft_group_id_lst = []
    for x in draft_groups:
        if x.contest_type_id == contest_type_id:
            draft_group_id_lst.append(x.draft_group_id)
            
    return min(draft_group_id_lst)

draft_group_id = get_draft_group_id(contests)

In [15]:
def get_draftables(draft_group_id):
    df_lst = []
    player_lst = Client().draftables(draft_group_id=draft_group_id).players
    for player in tqdm(player_lst):
        name_details = player.name_details
        first = name_details.first
        last = name_details.last
        name = f'{last}, {first}'
        salary = player.salary

        df = pd.DataFrame({
            'player_name': name,
            'showdown_salary': salary
        }, index=[0])

        df_lst.append(df)

    return pd.concat(df_lst).reset_index(drop=True)

In [16]:
draftable_df = get_draftables(draft_group_id)
draftable_df.to_csv('draftables.csv', index=False)

In [17]:
draft_group_id

54136

In [18]:
MY_API_KEY = 'dac62093607b097430db71962680'
API_PATH = 'https://feeds.datagolf.com/'
FILE_FORMAT = 'json'

In [19]:
tour_lst = [
    'PGA',
    'OPP',
    'EUR',
    'KFT',
]

In [20]:
config = {
    'field_updates':
    [
        'dg_id',
        'player_name',
        'country',
        'event_name',
        'current_round',
        'dk_id',
        'dk_salary',
        'fd_id',
        'fd_salary',
        'yh_id',
        'yh_salary',
        'early_late',
        'last_updated',
    ],
    'rankings':
    [
        'dg_id',
        #'player_name',
        'primary_tour',
        'datagolf_rank',
        'owgr_rank',
        'dg_skill_estimate'
    ],
    'pre_tourney_preds':
    [
        'dg_id',
        #'player_name',
        'make_cut_baseline',
        'top_10_baseline', 
        'top_20_baseline', 
        'top_5_baseline',
        'win_baseline', 
        'make_cut_baseline_history_fit',
        'top_10_baseline_history_fit', 
        'top_20_baseline_history_fit',
        'top_5_baseline_history_fit', 
        'win_baseline_history_fit'
    ],
    'skill_decomps':
    ['dg_id',
     #'player_name',
     'baseline_pred', 
     'timing_adjustment',
     'age_adjustment',
     #'cf_approach_comp',
     #'cf_short_comp',
     'strokes_gained_category_adjustment',
     'course_experience_adjustment',
     'course_history_adjustment', 
     'total_course_history_adjustment', 
     'total_fit_adjustment',
     'driving_accuracy_adjustment',
     'driving_distance_adjustment', 
     'other_fit_adjustment',
     'final_pred',
    ]
}

In [21]:
def get_field_updates(path, tour, key=MY_API_KEY):
    response = requests.get(f'{API_PATH}{path}?tour={tour}&key={key}')
    data = response.json()
    df = pd.DataFrame(data['field'])
    
    for col in data.keys():
        if col != 'field':
            df[col] = data[col]
    
    return df[config['field_updates']]

def get_rankings(path, key=MY_API_KEY):
    response = requests.get(f'{API_PATH}{path}?&key={key}')
    data = response.json()
    df = pd.DataFrame(data['rankings'])
    
    return df[config['rankings']]

def get_pre_tourney_preds(path, tour, odds_format='percent', key=MY_API_KEY):
    response = requests.get(f'{API_PATH}{path}?tour={tour}&odds_format={odds_format}&key={key}')
    data = response.json()
    
    model_dict = {}
    for model in data['models_available']:
        df = pd.DataFrame(data[model])
        pred_cols = ['make_cut','top_10','top_20','top_5','win']
        df.rename(
            columns={x: x + f'_{model}' for x in pred_cols}, inplace=True
        )
        
        model_dict[model] = df
        
    cols_to_use = [col for col in model_dict['baseline_history_fit'].columns if 'history' in col] + ['dg_id']
    
    df = (model_dict['baseline']
          .merge(model_dict['baseline_history_fit'][cols_to_use],
                 how='left',
                 on='dg_id'
                )
         )
    
    return df[config['pre_tourney_preds']]

def get_player_skill_decomps(path, tour, key=MY_API_KEY):
    response = requests.get(f'{API_PATH}{path}?tour={tour}&key={key}')
    data = response.json()
    
    df = pd.DataFrame(data['players'])
    
    return df[config['skill_decomps']]

In [22]:
def merge_dfs(dfs):
    merged_dfs = reduce(lambda left, right: pd.merge(left, right, on='dg_id', how='left'), dfs)
    
    return merged_dfs

In [23]:
field_updates_df = get_field_updates('field-updates', 'pga')
rank_df = get_rankings('preds/get-dg-rankings')
pred_data = get_pre_tourney_preds('preds/pre-tournament', 'pga')
skill_df = get_player_skill_decomps('preds/player-decompositions?', 'pga')

In [24]:
merged_df = merge_dfs([field_updates_df, rank_df, pred_data, skill_df])

In [25]:
now = datetime.now()
merged_df.to_csv(f'data_golf_{now.strftime("%Y-%m-%d")}.csv', index=False)

In [52]:
player_name_map = {
    'Lee, Kyoung-Hoon':'Lee, Kyounghoon',
    'Fitzpatrick, Matt':'Fitzpatrick, Matthew'
}

In [53]:
draftable_df['player_name'] = (np.where(draftable_df['player_name'].isin(player_name_map.keys()),
                                        draftable_df['player_name'].map(player_name_map),
                                        draftable_df['player_name'])
                              )

In [54]:
showdown_df = merged_df.merge(draftable_df, how = 'left', on ='player_name')